In [1]:
import pandas as pd
import numpy as np
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, ForeignKey, ForeignKeyConstraint

from sqlalchemy import Table, MetaData
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import Executable, ClauseElement
from sqlalchemy.sql import text
from sqlalchemy_views import CreateView, DropView

In [2]:
filepath = "db.sqlite3"
engine = create_engine(f"sqlite:///{filepath}", echo=True)
sqlite_connection = engine.connect()

2021-11-09 20:31:23,951 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-11-09 20:31:23,960 INFO sqlalchemy.engine.base.Engine ()
2021-11-09 20:31:23,966 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-11-09 20:31:23,970 INFO sqlalchemy.engine.base.Engine ()


In [4]:
df = pd.read_csv('music_characteristics_dataset.csv')
df

,id,genre,track_name,artist_name,popularity,valence,energy,danceability,acousticness,tempo,speechiness,mode,instrumentalness
0,7GILkDXz9y9x5KPHBKdrHA,acoustic,Spring för livet,Sara Varga,0,0.5980,0.572,0.786,0.2930,119.943,0.0590,1,0.000000
1,2xwsf9FuFINP1X4FTsqZ7Q,acoustic,Peace,O.A.R.,62,0.4930,0.721,0.449,0.1660,178.141,0.0392,1,0.000000
2,4VJgrWjrkodaGiq3xKz62z,acoustic,Sometimes (I Wish),City and Colour,40,0.1350,0.467,0.523,0.3110,129.953,0.0281,0,0.000020
3,46XMysg4VurmyAQ28tshqz,acoustic,The Mess I Made,Parachute,47,0.1730,0.591,0.405,0.0153,165.377,0.0339,1,0.000000
4,25J4d78ESH2MgAJoIakB8O,acoustic,Lost In The Light,Bahamas,50,0.2460,0.267,0.597,0.7930,75.535,0.0309,1,0.001400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,3CCGsOj2efotChiScMmCWg,world-music,A Minha Menina,Os Mutantes,0,0.4290,0.930,0.455,0.0568,91.410,0.1260,1,0.000000
3001,6bYpEZmbTldBmQtnnfDRDk,world-music,Raoui,Various Artists,0,0.6250,0.058,0.560,0.9260,107.658,0.0500,0,0.000003
3002,1xJtdt697VaDWFJ1eEwlPO,world-music,Cat Bed Music,Pet Music World,5,0.0394,0.033,0.304,0.9920,71.282,0.0655,1,0.963000
3003,1Nov1m4HA8PBs2vyWjp2Cv,world-music,Mas Que Nada,Jorge Ben Jor,0,0.5980,0.403,0.514,0.6720,89.672,0.0325,0,0.000366


In [5]:
# Create df with only author and author_id

# imdb = df.copy().sort_values('AUTHOR', ascending=True)
# author = imdb["AUTHOR"]
# author = pd.DataFrame(author)
# author = author.drop_duplicates(subset="AUTHOR", keep="first")
# author["AUTHOR_ID"] = author.reset_index().index
# authors = author.sort_values('AUTHOR_ID', ascending=True)
# authors = authors.set_index('AUTHOR_ID')
# authors

In [6]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Create basic structure of authors table
class Song_Tracks(Base):
    __tablename__ = 'song_tracks'
    id = Column(Integer, primary_key=True)
    genre = Column(String)
    track_name = Column(String)
    artist_name = Column(String)
    popularity = Column(Integer)
    track_name = Column(String)
    valence = Column(Float)
    energy = Column(Float)
    danceability = Column(Float)
    acousticness = Column(Float)
    tempo = Column(Float)
    speechiness = Column(Float)
    mode = Column(Integer)
    instrumentalness = Column(Float)
    


In [7]:
# Convert authors df to sqlite table

sqlite_table = "song_tracks"
df.to_sql(sqlite_table, sqlite_connection, if_exists='append')

2021-11-09 20:36:59,715 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("song_tracks")
2021-11-09 20:36:59,717 INFO sqlalchemy.engine.base.Engine ()
2021-11-09 20:36:59,724 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("song_tracks")
2021-11-09 20:36:59,728 INFO sqlalchemy.engine.base.Engine ()
2021-11-09 20:36:59,739 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE song_tracks (
	"index" BIGINT, 
	id TEXT, 
	genre TEXT, 
	track_name TEXT, 
	artist_name TEXT, 
	popularity BIGINT, 
	valence FLOAT, 
	energy FLOAT, 
	danceability FLOAT, 
	acousticness FLOAT, 
	tempo FLOAT, 
	speechiness FLOAT, 
	mode BIGINT, 
	instrumentalness FLOAT
)


2021-11-09 20:36:59,741 INFO sqlalchemy.engine.base.Engine ()
2021-11-09 20:36:59,755 INFO sqlalchemy.engine.base.Engine COMMIT
2021-11-09 20:36:59,758 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_song_tracks_index ON song_tracks ("index")
2021-11-09 20:36:59,760 INFO sqlalchemy.engine.base.Engine ()
2021-11-09 20:36:59,770 INFO

In [8]:
sqlite_connection.close()